In [1]:
# %conda install -c conda-forge ta-lib -q
# %pip install -r requirements.txt -q

# import boto3
import data
import glob
import matplotlib.pyplot as plt
import models
import numpy as np
import os
import pandas as pd
import pypickle
import strategy

from data import * 
from feature_engineering import create_features, normalize_features
from io import StringIO
from IPython.display import display
from sklearn.linear_model import LassoCV
from utils import log

2025-02-21 22:13:58.759163: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-21 22:13:58.769194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740147238.781254   12891 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740147238.784955   12891 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-21 22:13:58.797598: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Read raw data

In [2]:
# Define S3 bucket and file path
bucket_name = 'mscfe-capstone-project-bucket'
file_key = 'filtered_data.csv'

filtered = pd.read_csv('filtered_data.csv').drop(['Unnamed: 0'], axis=1)
display(filtered)

,seriesX,seriesY,correlation,spread_adf_stat,spread_adf_pValue,spread_stationary,series_coint_stat,series_coint_pValue,series_cointegrated
0,GOOG,GOOGL,0.999931,-4.971937,0.000025,True,-4.976339,0.000182,True
1,META,DAXEX_ETF,0.888934,-2.917056,0.043400,True,-3.483091,0.033924,True
2,AMZN,BNBUSDT,0.877059,-3.631203,0.005192,True,-3.545100,0.028601,True
3,AMZN,TRXUSDT,0.814577,-3.869803,0.002269,True,-3.387413,0.043768,True
4,SPY_ETF,AVAXUSDT,0.847292,-3.309927,0.014441,True,-3.698512,0.018406,True
5,SPY_ETF,ETHUSDT,0.862483,-3.428205,0.010034,True,-3.737367,0.016394,True
6,EQQQ_ETF,BNBUSDT,0.886060,-3.816404,0.002744,True,-3.670509,0.019987,True
7,BNBUSDT,ADAUSDT,0.901459,-3.279504,0.015820,True,-3.775562,0.014607,True
8,BNBUSDT,BTCUSDT,0.884604,-2.909961,0.044202,True,-3.393160,0.043116,True
9,BNBUSDT,TRXUSDT,0.933562,-3.656380,0.004770,True,-4.003675,0.007095,True


In [3]:
print('********************')
pairs = filtered['seriesX'].iloc[1:] + ' ' + filtered['seriesY'].iloc[1:]
print('Trading pairs:')
for p in pairs:
    print(p)
print('********************')

unique_tickers = pd.concat([filtered['seriesX'].iloc[1:], filtered['seriesY'].iloc[1:]]).drop_duplicates()
print('Unique tickers:')
for ut in unique_tickers:
    print(ut)
print('********************')

# Preview the source data
file_key = 'df_full.csv'
df_full = pd.read_csv('df_full.csv').drop(['Unnamed: 0'], axis=1)
# obj = s3.get_object(Bucket=bucket_name, Key=file_key)
# df_full = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8'))).drop(['Unnamed: 0'], axis=1)
display(df_full.sample(5))

********************
Trading pairs:
META DAXEX_ETF
AMZN BNBUSDT
AMZN TRXUSDT
SPY_ETF AVAXUSDT
SPY_ETF ETHUSDT
EQQQ_ETF BNBUSDT
BNBUSDT ADAUSDT
BNBUSDT BTCUSDT
BNBUSDT TRXUSDT
BNBUSDT XRPUSDT
********************
Unique tickers:
META
AMZN
SPY_ETF
EQQQ_ETF
BNBUSDT
DAXEX_ETF
TRXUSDT
AVAXUSDT
ETHUSDT
ADAUSDT
BTCUSDT
XRPUSDT
********************


,OpenTime,Open,High,Low,Close,Symbol
356649,2025-01-08 04:50:00,96078.13,96078.13,96009.07,96009.07,BTCUSDT
346279,2025-01-04 10:30:00,98057.27,98079.31,97976.74,98067.81,BTCUSDT
273418,2024-12-10 03:20:00,698.71,700.05,698.70,700.00,BNBUSDT
262061,2024-12-06 04:40:00,3875.77,3878.60,3865.00,3865.31,ETHUSDT
48795,2024-12-10 11:20:00,73.00,73.00,73.00,73.00,CAC40_ETF


# Features Engineering Config

In [4]:
config = {
    'ema': [8, 21, 55],
    'macd': {
        'fast': 12,
        'slow': 26,
        'signal': 9,
    },
    'rsi': [14],
    'bb': {
        'timeperiod': 20,
        'nbdevup': 2,
        'nbdevdn': 2,
    },
    'atr': {
        'timeperiod': 14,
    },
    'stoch': {
        'fastk_period': 14,
        'slowk_period': 3,
        'slowd_period': 3,
    },
    'cci': {
        'timeperiod': 20,
    },
    'willr': {
        'timeperiod': 14,
    },
}

# Hyperparameters Tunning

## Ridge Regression

In [ ]:
untuned_performance = []
scalers = []
validation_dfs = pd.DataFrame()
s = 2
tuned_params = []

verbose = False

# for p in pairs:
for p in pairs:

    print(f"Pairs: {p}")
    
    tickerX, tickerY = p.split(' ')    
    spread_df = create_features(tickerX, tickerY,
                                df_full[df_full['Symbol']==tickerX],
                                df_full[df_full['Symbol']==tickerY],
                                config=config,
                                dropna=True)
    
    spread_df, scaler = normalize_features(spread_df)
    spread_df = spread_df.rename(columns={'Spread':'NormalizedSpread'})
    # Save scaler for test set
    scalers.append({p:scaler})

    # Preview the data
    # display(spread_df.head())
    # Added .copy() to avoid warnings
    model_df = spread_df.copy()
    
    # Perform LASSO feature selection
    X_temp = model_df.drop('NormalizedSpread', axis=1)
    y_temp = model_df['NormalizedSpread']
    lasso = LassoCV(cv=5, random_state=42)
    lasso.fit(X_temp, y_temp)
    # Get selected features based on non-zero coefficients
    selected_features = X_temp.columns[abs(lasso.coef_) > 0].tolist()
    model_df = pd.concat([X_temp[selected_features], y_temp], axis=1)

    param_ridge = {
        'alpha': [0.001, 0.01, 0.1, 1, 10, 100],
        'solver': ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag'],
    }

    best_model, best_params, best_mse, best_df_test = models.hyperparameter_tuning(
        df=model_df, p=p, model_func=models.ridge_regression, param_grid=param_ridge, verbose=verbose)

    stats = {
        'pair':p, 
        'mse_training': best_mse,
        'data_points': len(model_df),
        'features': len(model_df.columns),
    }
    stats.update(best_params)
    tuned_params.append(stats)

In [ ]:
ridge_regression_results = pd.DataFrame(tuned_params)
ridge_regression_results['algorithm']  = 'ridge_regression'
ridge_regression_results

## Xgboost Regression

In [5]:
untuned_performance = []
scalers = []
validation_dfs = pd.DataFrame()
s = 2
tuned_params = []

verbose = True

for p in pairs:

    print(f"Pairs: {p}")
    
    tickerX, tickerY = p.split(' ')    
    spread_df = create_features(tickerX, tickerY,
                                df_full[df_full['Symbol']==tickerX],
                                df_full[df_full['Symbol']==tickerY],
                                config=config,
                                dropna=True)
    
    spread_df, scaler = normalize_features(spread_df)
    spread_df = spread_df.rename(columns={'Spread':'NormalizedSpread'})
    # Save scaler for test set
    scalers.append({p:scaler})

    # Preview the data
    # display(spread_df.head())
    # Added .copy() to avoid warnings
    model_df = spread_df.copy()
    
    # Perform LASSO feature selection
    X_temp = model_df.drop('NormalizedSpread', axis=1)
    y_temp = model_df['NormalizedSpread']
    lasso = LassoCV(cv=5, random_state=42)
    lasso.fit(X_temp, y_temp)
    # Get selected features based on non-zero coefficients
    selected_features = X_temp.columns[abs(lasso.coef_) > 0].tolist()
    model_df = pd.concat([X_temp[selected_features], y_temp], axis=1)

    param_xgboost = {
        'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Step size shrinkage
        'n_estimators': [50, 100, 200],  # Number of boosting rounds
        'max_depth': [5, 7, 9],  # Maximum tree depth
    }

    best_model, best_params, best_mse, best_df_test = models.hyperparameter_tuning(
        df=model_df, p=p, model_func=models.xgboost_regression, param_grid=param_xgboost, verbose=verbose)

    stats = {
        'pair':p, 
        'mse_training': best_mse,
        'data_points': len(model_df),
        'features': len(model_df.columns),
    }
    stats.update(best_params)
    tuned_params.append(stats)

Pairs: META DAXEX_ETF
Testing params: {'learning_rate': 0.01, 'n_estimators': 50, 'max_depth': 5}


/home/alfred/miniconda3/envs/tf_py311/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [22:14:01] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


XGBoost MSE: 0.22917819491572797
Testing params: {'learning_rate': 0.01, 'n_estimators': 50, 'max_depth': 7}
XGBoost MSE: 0.22995117397697093
Testing params: {'learning_rate': 0.01, 'n_estimators': 50, 'max_depth': 9}
XGBoost MSE: 0.22973271748518237
Testing params: {'learning_rate': 0.01, 'n_estimators': 100, 'max_depth': 5}
XGBoost MSE: 0.08905524412554593
Testing params: {'learning_rate': 0.01, 'n_estimators': 100, 'max_depth': 7}
XGBoost MSE: 0.0887607001566571
Testing params: {'learning_rate': 0.01, 'n_estimators': 100, 'max_depth': 9}
XGBoost MSE: 0.0895128851140909
Testing params: {'learning_rate': 0.01, 'n_estimators': 200, 'max_depth': 5}
XGBoost MSE: 0.030358555439645125
Testing params: {'learning_rate': 0.01, 'n_estimators': 200, 'max_depth': 7}
XGBoost MSE: 0.029320593475696502
Testing params: {'learning_rate': 0.01, 'n_estimators': 200, 'max_depth': 9}
XGBoost MSE: 0.030567664437871463
Testing params: {'learning_rate': 0.05, 'n_estimators': 50, 'max_depth': 5}
XGBoost MSE:

/home/alfred/miniconda3/envs/tf_py311/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.2822332811043111, tolerance: 0.24066466603867667
  model = cd_fast.enet_coordinate_descent_gram(
/home/alfred/miniconda3/envs/tf_py311/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:681: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.3927876898957763, tolerance: 0.24066466603867667
  model = cd_fast.enet_coordinate_descent_gram(
/home/alfred/miniconda3/envs/tf_py311/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.187e+00, tolerance: 4.997e-01
  model =

XGBoost MSE: 1.2099121332582972
Testing params: {'learning_rate': 0.01, 'n_estimators': 50, 'max_depth': 7}
XGBoost MSE: 0.9897419762292147
Testing params: {'learning_rate': 0.01, 'n_estimators': 50, 'max_depth': 9}
XGBoost MSE: 0.9670633868422445
Testing params: {'learning_rate': 0.01, 'n_estimators': 100, 'max_depth': 5}
XGBoost MSE: 0.9231097629119633
Testing params: {'learning_rate': 0.01, 'n_estimators': 100, 'max_depth': 7}
XGBoost MSE: 0.7025155387331136
Testing params: {'learning_rate': 0.01, 'n_estimators': 100, 'max_depth': 9}
XGBoost MSE: 0.6703123185893781
Testing params: {'learning_rate': 0.01, 'n_estimators': 200, 'max_depth': 5}
XGBoost MSE: 0.5801011311652275
Testing params: {'learning_rate': 0.01, 'n_estimators': 200, 'max_depth': 7}
XGBoost MSE: 0.46823643942935644
Testing params: {'learning_rate': 0.01, 'n_estimators': 200, 'max_depth': 9}
XGBoost MSE: 0.4691531346527904
Testing params: {'learning_rate': 0.05, 'n_estimators': 50, 'max_depth': 5}
XGBoost MSE: 0.518772

In [ ]:
xgboost_regression_results = pd.DataFrame(tuned_params)
xgboost_regression_results['algorithm']  = 'xgboost_regression'
xgboost_regression_results

In [ ]:
spread_df.TradingSignal.value_counts()

In [ ]:
spread_df

In [ ]:
best_df_test

In [ ]:
model_df